In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
owd = os.getcwd()
os.chdir('../src/features')
import preprocess as pp
os.chdir(owd)
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
import statsmodels.api as sm

In [2]:
conn = sqlite3.connect('../src/data/td_V2.db')
git_commits = pd.read_sql_query("SELECT * FROM GIT_COMMITS",conn)
szz_fault_inducing_commits = pd.read_sql_query("SELECT * FROM szz_fault_inducing_commits",conn)
refactoring_miner = pd.read_sql_query("SELECT * FROM refactoring_miner",conn)
refactoring_miner = refactoring_miner[refactoring_miner["COMMIT_HASH"].isin(git_commits["COMMIT_HASH"])]
git_commits_changes = pd.read_sql_query("SELECT * FROM GIT_COMMITS_CHANGES", conn)
git_commits_changes = git_commits_changes[git_commits_changes["COMMIT_HASH"].isin(refactoring_miner["COMMIT_HASH"])]

In [3]:
print(len(refactoring_miner), len(szz_fault_inducing_commits))
clean_df = pp.preprocess(git_commits, szz_fault_inducing_commits, refactoring_miner, git_commits_changes)
clean_df = clean_df[clean_df["FAULT_INDUCING_COMMIT_HASH"].isin(szz_fault_inducing_commits["FAULT_INDUCING_COMMIT_HASH"])]
clean_df

32334 52428


/Users/paulozanogarcia/Documents/UNIVERSITAT/4t/TAED/Technical-Debt-DLMP/src/features/preprocess.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  git_commits_changes["LINES_ADDED"] = git_commits_changes["LINES_ADDED"].astype(int)
/Users/paulozanogarcia/Documents/UNIVERSITAT/4t/TAED/Technical-Debt-DLMP/src/features/preprocess.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  git_commits_changes["LINES_REMOVED"] = git_commits_changes["LINES_REMOVED"].astype(int)


,REFACTORING_TYPE,FAULT_INDUCING_COMMIT_HASH,PROJECT_ID,COMMIT_MESSAGE,COMMITTER_DATE,BRANCHES,COMMITTER_ID,LINES_ADDED,LINES_REMOVED
0,Move Class,d522432b79044740831a132d8b92e7dab5477444,org.apache:bcel,Fix messed up package rename git-svn-id: http...,2016-06-07 17:28:43+00:00,{'master'},Benedikt Ritter,1506,1505
1,Extract Class,02f53e4fa6ec7fa339c8c9273183669d7f5928c9,org.apache:bcel,Gradually working towards restoring binary com...,2015-09-11 00:30:19+00:00,{'master'},Sebastian Bazley,3915,2105
2,Move Method,02f53e4fa6ec7fa339c8c9273183669d7f5928c9,org.apache:bcel,Gradually working towards restoring binary com...,2015-09-11 00:30:19+00:00,{'master'},Sebastian Bazley,3915,2105
3,Move Attribute,02f53e4fa6ec7fa339c8c9273183669d7f5928c9,org.apache:bcel,Gradually working towards restoring binary com...,2015-09-11 00:30:19+00:00,{'master'},Sebastian Bazley,3915,2105
4,Extract And Move Method,cfbd46ac5bfbb669776e9403375a581fc4d4e8bd,org.apache:bcel,Prepare for eventual privatisation of other ar...,2015-08-25 18:45:21+00:00,{'master'},Sebastian Bazley,177,29
...,...,...,...,...,...,...,...,...,...
3021,Move Class,0096b20883acf08c46389c8d232a49d8886074dc,org.apache:thrift,THRIFT-366. java: Lots of little warnings in j...,2009-03-13 00:28:06+00:00,{'master'},Bryan Duxbury,99,146
3022,Move Source Folder,249d7cb199b5c08e7a7a51189a733cc8fef12cf9,org.apache:thrift,THRIFT-166. java: Java tests should be in lib/...,2009-01-29 01:21:20+00:00,{'master'},Bryan Duxbury,52,65
3023,Move Class,afa80ea73d4cbf83afee807e5d9f9034fa177ade,org.apache:thrift,THRIFT-34 Java libraries should be namespaced ...,2009-01-15 23:47:51+00:00,{'master'},Bryan Duxbury,249,249
3024,Extract Method,d92ca7dcea105837e3e576f797501c19929eced5,org.apache:thrift,Prevent Java's TProtocolUtil.skip from overflo...,2008-03-18 19:32:00+00:00,{'master'},David Reiss,48,6


In [4]:
(clean_df["FAULT_INDUCING_COMMIT_HASH"] == "30cff66e5efe6b99b7e7a2293061f0eea0def02f").sum()

12

In [5]:
szz_fault_inducing_commits[szz_fault_inducing_commits.FAULT_INDUCING_COMMIT_HASH == 'fc186f50392bf8c804b1d314971c92efa5f52fc3']

,PROJECT_ID,FAULT_FIXING_COMMIT_HASH,FAULT_INDUCING_COMMIT_HASH
4095,org.apache:cayenne,20616104fe25faaf014b23180fb8a40a6d93cbad,fc186f50392bf8c804b1d314971c92efa5f52fc3
4096,org.apache:cayenne,770a960d251b42dc323a87b2cdf4cef844777a31,fc186f50392bf8c804b1d314971c92efa5f52fc3
4097,org.apache:cayenne,6044e64c46033377d72c5cf1b3342b376df0cee7,fc186f50392bf8c804b1d314971c92efa5f52fc3
4101,org.apache:cayenne,5f364c585b9fc39e02d271747f30447c62030ab2,fc186f50392bf8c804b1d314971c92efa5f52fc3
4102,org.apache:cayenne,fa5f7734aa45a448463d44ec7474326f978710c1,fc186f50392bf8c804b1d314971c92efa5f52fc3
...,...,...,...
7039,org.apache:cayenne,9e6c27ba798839f5dd3e26f36e89b8bb8bc297d7,fc186f50392bf8c804b1d314971c92efa5f52fc3
7046,org.apache:cayenne,9b91f6d2a150d83918acf8999442f2bb120f2ce6,fc186f50392bf8c804b1d314971c92efa5f52fc3
7050,org.apache:cayenne,90c19b0c18614346eb110265da2780caaa84dc09,fc186f50392bf8c804b1d314971c92efa5f52fc3
7068,org.apache:cayenne,fa4bddd4c1f6081f4d4f604f5c19826416387244,fc186f50392bf8c804b1d314971c92efa5f52fc3


In [6]:
refactor_commits = refactoring_miner[["COMMIT_HASH", "REFACTORING_TYPE"]].merge(git_commits_changes[["COMMIT_HASH", "DATE", "COMMITTER_ID", "LINES_ADDED", "LINES_REMOVED"]], on = "COMMIT_HASH")
refactor_commits["LINES_ADDED"] = refactor_commits["LINES_ADDED"].astype(int)
refactor_commits["LINES_REMOVED"] = refactor_commits["LINES_REMOVED"].astype(int)
refactor_commits = refactor_commits.groupby(['COMMIT_HASH', 'COMMITTER_ID', 'REFACTORING_TYPE']).sum().reset_index()

In [7]:
inducing_ones = refactor_commits[refactor_commits["COMMIT_HASH"].isin(clean_df["FAULT_INDUCING_COMMIT_HASH"])]
refactor_commits["LABEL"] = np.where(refactor_commits["COMMIT_HASH"].isin(clean_df["FAULT_INDUCING_COMMIT_HASH"]), 1, 0)
y = refactor_commits["LABEL"]
X = refactor_commits.drop(columns = ["COMMIT_HASH","LABEL", "COMMITTER_ID"])
X["REFACTORING_TYPE"] = X["REFACTORING_TYPE"].astype("category").cat.codes

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC(kernel = 'rbf')
clf.fit(X_train, y_train)
predicts = clf.predict(X_test)
confusion_matrix(y_test, predicts)

array([[2503,    8],
       [ 976,   40]])

In [ ]:
y_train = pd.Series(y_train).replace(1,0.95).replace(0,0.05)
y_train = np.asarray(y_train).astype(float)
model = sm.GLM(y_train,X_train, family = sm.families.Binomial())
res = model.fit()
res = res.predict(X_test)
confusion_matrix(y_test)